# Segmenting and Clustering Neighborhoods in Toronto Part I.

In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:


### Scraping Canada Postal Codes from Wikipidea

In [1]:
# Import Libraries
import pandas as pd
import os,sys
import urllib
import requests 
from urllib.request import urlopen
from bs4 import BeautifulSoup

Link to wikipedia List of postal codes of Canada: M. Postal codes beginning with M are located within the city of Toronto in the province of Ontario.

### Download the table data

In [2]:
# Link to wikipedia and Download
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urlopen(URL)
soup = BeautifulSoup(page, "lxml")
page.close()
 
fp = open("data.csv","w")
tables = soup.findAll('table')
tab = tables[0]

# Find each rows
for tr in tab.tbody.findAll('tr'):
    #Columns
    for th in tr.findAll('th'):
        text = th.getText().strip()+','
        fp.write(text)
    for td in tr.findAll('td'):
        text = td.getText().strip()+','
        fp.write(text)
    fp.write('\n')
fp.close()

### Create a Dataframe

In [3]:
toronto_df = pd.read_csv('data.csv')
toronto_df.drop('Unnamed: 3',axis=1,inplace = True)
toronto_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [4]:
toronto_df.rename(columns={'Neighbourhood': 'Neighborhood'}, inplace=True)
toronto_df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [5]:
# View data shape
toronto_df.shape

(289, 3)

### Identify and Clean "Not assigned" Value in the "Borough" column

In [6]:
toronto_df_2 = toronto_df[ ~ toronto_df['Borough'].str.contains('Not assigned')]
toronto_df_2.head()

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [7]:
toronto_df_2.shape

(212, 3)

In [8]:
# Check Postcode M5A Neighborhood
toronto_df_2.loc[toronto_df_2['Postcode'] == "M5A"]

,Postcode,Borough,Neighborhood
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park


###  Combine and Grouping Dataframe with same Postcode on Comma Separated Neighborhood

In [9]:
group = toronto_df_2.groupby('Postcode')
grouped_neighborhoods = group['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
grouped_boroughs = group['Borough'].apply(lambda x: set(x).pop())
toronto_df_final = pd.DataFrame(list(zip(grouped_boroughs.index, grouped_boroughs, grouped_neighborhoods)))
toronto_df_final.columns = ['Postcode', 'Borough', 'Neighborhood']
toronto_df_final.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [10]:
# Check combine result on M5A postcode
toronto_df_final.loc[toronto_df_final['Postcode'] == "M5A"]

,Postcode,Borough,Neighborhood
53,M5A,Downtown Toronto,"Harbourfront, Regent Park"


In [11]:
# Save dataframe for future use
toronto_df_final.to_csv('toronto_part1.csv')

In [12]:
toronto_df_final.shape

(103, 3)